In [1]:
import pandas as pd
import numpy as np
import warnings

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [2]:
data = pd.read_csv(path+'data/ml_input.csv')
processed = pd.read_csv(path+'data/processed.csv')
parameters = pd.read_csv(path+'parameters/rf_regressor.csv')

In [3]:
### Season to test results

N = 2021

In [4]:
params_to_drop = ['season', 'round', 'podium', 'driver_points_from']

In [5]:
df = data.copy()

train = df[df.season < N]

scaler = StandardScaler()

X_train = train.drop(params_to_drop, axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

y_train = np.asarray(train.driver_points_from.values)

In [6]:
def get_predictions(X_train, y_train, model):
    points = []
    winners = []
    
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(params_to_drop, axis=1)
        y_test = test.driver_points_from

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict(X_test), columns=['predicted_points'])
        prediction_df['actual_points'] = y_test.reset_index(drop=True)

        prediction_df['predicted_winner'] = prediction_df.predicted_points.map(lambda x: 1 if x == prediction_df.predicted_points.max() else 0)
        prediction_df['actual_winner'] = prediction_df.actual_points.map(lambda x: 1 if x == prediction_df.actual_points.max() else 0)

        points += list(prediction_df.predicted_points.values)
        winners += list(prediction_df.predicted_winner.values)

        # Retrain model
        X_train = pd.concat([X_train, X_test])
        y_train = np.append(y_train, y_test)

        model.fit(X_train, y_train)
        

    return points, winners

In [7]:
parameters.head()

,model,criterion,max_features,max_depth,score
0,random_forest_regressor,squared_error,0.8,5,0.578947
1,random_forest_regressor,squared_error,0.8,41,0.578947
2,random_forest_regressor,squared_error,0.8,47,0.578947
3,random_forest_regressor,squared_error,0.8,55,0.578947
4,random_forest_regressor,squared_error,0.8,7,0.578947


In [8]:
params = parameters.iloc[0]
criterion = params.criterion
max_features = params.max_features	
max_depth = params.max_depth

params

model           random_forest_regressor
criterion                 squared_error
max_features                        0.8
max_depth                             5
score                          0.578947
Name: 0, dtype: object

In [9]:
# Random Forest Regressor

test = processed.copy()
test = test[['season', 'round', 'driver', 'starting_grid', 'podium', 'driver_points_from', 'q_delta']].query('season ==@N')

model_params = (criterion, max_features, max_depth)
model = RandomForestRegressor(criterion=criterion, max_features=max_features, max_depth=max_depth)
model.fit(X_train, y_train)

points, winner = get_predictions(X_train, y_train, model)

test['predicted_points'] = points
test['predicted_winner'] = winner

In [10]:
test.query('predicted_winner == 1')

,season,round,driver,starting_grid,podium,driver_points_from,q_delta,predicted_points,predicted_winner
2743,2021,1,max_verstappen,1,2,18.0,0.000,16.750675,1
2763,2021,2,lewis_hamilton,1,2,19.0,0.000,19.639005,1
2781,2021,3,lewis_hamilton,2,1,25.0,0.007,15.777014,1
2801,2021,4,lewis_hamilton,1,1,25.0,0.000,19.445427,1
2839,2021,5,charles_leclerc,1,20,0.0,0.000,17.323889,1
2853,2021,6,lewis_hamilton,2,15,0.0,0.000,17.988800,1
2858,2021,7,max_verstappen,1,1,26.0,0.000,19.839999,1
2876,2021,8,max_verstappen,1,1,25.0,0.000,19.695450,1
2896,2021,9,max_verstappen,1,1,26.0,0.000,19.186222,1
2917,2021,11,lewis_hamilton,1,2,18.0,0.000,19.797953,1


In [11]:
rounds = len(test['round'].unique())
correct = len(test.query('predicted_winner == 1 & podium <= 3'))

(correct / rounds) * 100

84.21052631578947